In [5]:
import os
import sys
import numpy as np
import pandas as pd
import math as mth

cdfPath = "E:\Github\Ladder-Iterative-Loadflow\Documentations\Datasets\IEEE 33 CDF (Updated)_2.xlsx"
tol = 0.001
readInput = pd.read_excel(cdfPath, header=None)

numpyConversion = readInput.to_numpy()
checker = checkIfAllTablesExist(readInput)

#SBase = (parser.SBaseExtractor(numpyConversion)/(mth.sqrt(3)))*1000
SBase = (SBaseExtractor(numpyConversion))

if checker == False:
    raise ValueError("Excel does not contain Bus or Branch data, please double check and try again")

busData = extractData(numpyConversion, "BUS DATA FOLLOWS")
branchData = extractData(numpyConversion, "BRANCH DATA FOLLOWS")
VBase = busData[0, 3]

sortedBranchData = branchReorder(branchData)

print("Table extraction complete")

#calcMain.calcMain(busData, sortedBranchData, float(tol), SBase, VBase)

Table extraction complete


In [7]:
import numpy as np
import pandas as pd
import sys
import os

def checkIfAllTablesExist(arr):
    # Check if the excel file contains all the necessary tables
    if "BUS DATA FOLLOWS" or "BRANCH DATA FOLLOWS" in arr:
        return True
    else:
        return False

def extractData(npArr, txt):

    lenOfArray = npArr.shape[0] # Length of the array
    widthOfArray = npArr.shape[1] # Width of the array
    forLoopStartValue = int(np.where(npArr==txt)[0])+1 # The loop will start from the row after the specified text
    for i in range(forLoopStartValue, lenOfArray):
        if npArr[i][0] == -999: # Loop ends when the -999 is found
            newArray = npArr[forLoopStartValue:i] # This the table extracted
            break

    if txt == "BUS DATA FOLLOWS":
        arrayToReturn = np.zeros((newArray.shape[0],4))
        arrayToReturn[:,0] = newArray[:,0] # Bus number
        arrayToReturn[:,1] = newArray[:,7] # P
        arrayToReturn[:,2] = newArray[:,8] # Q
        arrayToReturn[:,3] = newArray[:,11] # V Base
    elif txt == "BRANCH DATA FOLLOWS":
        arrayToReturn = np.zeros((newArray.shape[0],4))
        arrayToReturn[:,0] = newArray[:,0] # From
        arrayToReturn[:,1] = newArray[:,1] # To
        arrayToReturn[:,2] = newArray[:,6] # R
        arrayToReturn[:,3] = newArray[:,7] # X
        
    return arrayToReturn

def branchReorder(branchData):
    newData = branchData
    lenOfArray = newData.shape[0]
    zerosCol = np.zeros((newData.shape[0],1))
    newData = np.c_[newData, zerosCol]
    newBranchDataArr = np.zeros((newData.shape[0],4))
    n=0
    duplicateRan = False
    
    i = 0
    while i < lenOfArray:
        
        dup = np.where(newData[:,0]==newData[i,0])
        if len(dup[0])==1 and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1            
        else:
            for j in dup[0]:
                k = j
                if (j > i) and (newData[k,4]==0):
                    
                    while(1):
                        if k+1 == lenOfArray:
                            if (newData[k,0]==newData[k-1,1]):
                                newBranchDataArr[n,0] = newData[k,0]
                                newBranchDataArr[n,1] = newData[k,1]
                                newBranchDataArr[n,2] = newData[k,2]
                                newBranchDataArr[n,3] = newData[k,3]
                                newData[k,4] = 1
                                n = n+1
                                k = k+1
                            break
                        if (newData[k,1]==newData[k+1,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                        elif (newData[k,1]!=newData[k+1,0]) and (newBranchDataArr[n-1,0] != newData[k,0]):
                            newBranchDataArr[n,0] = newData[k,0]
                            newBranchDataArr[n,1] = newData[k,1]
                            newBranchDataArr[n,2] = newData[k,2]
                            newBranchDataArr[n,3] = newData[k,3]
                            newData[k,4] = 1
                            n = n+1
                            k = k+1
                            break
                    duplicateRan = True
        if duplicateRan == True and (newData[i,4]==0):
            newBranchDataArr[n,0] = newData[i,0]
            newBranchDataArr[n,1] = newData[i,1]
            newBranchDataArr[n,2] = newData[i,2]
            newBranchDataArr[n,3] = newData[i,3]
            newData[i,4] = 1
            n=n+1
            duplicateRan = False
        
        i = i + 1
    return newBranchDataArr

def SBaseExtractor(arr):
    RyeFinder = arr[0][0]
    subString = "RYERSON UNIVERSITY"

    try:
        RyeFinder.index(subString)
    except ValueError:
        raise ValueError("Header with SBase is missing")
    else:
        splitRyeString = RyeFinder.split(" ")
        SBase = splitRyeString[4]
        return float(SBase)

In [10]:
import numpy as np

def FWR(inputArr, Vs):
    for i in range(len(inputArr)):
        ZZ = complex(inputArr[i,2], inputArr[i,3])
        dup = np.where(inputArr[:, 0] == inputArr[i, 0])
        if len(dup[0]) == 1:
            if i == 0:
                inputArr[i, 4] = Vs - (ZZ*inputArr[i, 6]) # voltage at node 2
             # If the current row is part of the previous row's branch
            elif inputArr[i, 0] == inputArr[i-1, 1]:
                inputArr[i, 4] = inputArr[i - 1, 4] - (ZZ*inputArr[i, 6]) # voltage at node
            #print(inputArr[i, 0])
         #else:
      # If the current row has a duplicate somewhere
        elif len(dup[0]) > 1:
         # Iterate through the entire list of duplicates
            for j in dup[0]:
            #print(dup[0])
            # If the current duplicate is also the current bus
                if j == i:
                  # Check if we are still part of the same branch
                    if inputArr[j, 0] == inputArr[j - 1, 1]:
                        inputArr[j, 4] = inputArr[j - 1, 4] - (ZZ*inputArr[j, 6]) # voltage at node
                     #print(inputArr[i, 0])
                     #print(j)
                    else:
                     # Pull the bus number of the first duplicate, to get the voltage at that point
                        k = dup[0][0]
                     # Verify that the duplicate's previous row is still part of same branch
                        if inputArr[k, 0] == inputArr[k - 1, 1]:
                            inputArr[j, 4] = inputArr[k - 1, 4] - (ZZ*inputArr[j, 6]) # voltage at node
                        #print(inputArr[j, 0])
                        #print(k)
                        #print(j)
                        #print("")


    return inputArr

In [11]:
import numpy as np
import math as mth

def BKWsweep(busArr, inputArr):
  # Generate the load currents
  for i in range(len(inputArr)-1,-1,-1):
      busFinder = np.where(busArr[:, 0] == inputArr[i, 1])
      j = busFinder[0][0]
      S = complex(busArr[j,1], busArr[j,2])
      inputArr[i, 5] = ((S)/inputArr[i, 4]).conjugate()
      #print(inputArr[i, 5])

  # Calculate the currents for each bus
  for i in range(len(inputArr)-1,-1,-1):
      dup = np.where(inputArr[:, 0] == inputArr[i, 1])
      #print(dup[0])
      if len(dup[0]) <= 1:
          if len(dup[0]) == 0: # If it is the last bus of the branch, duplicate will be empty
          #if i == len(inputArr) - 1:
              inputArr[i, 6] = inputArr[i, 5] + 0 # Final bus, so all current goes into load
              #print(inputArr[i, 1])
          elif inputArr[i, 1] == inputArr[i + 1, 0]:
              inputArr[i, 6] = inputArr[i, 5] + inputArr[i + 1, 6]
              #print(inputArr[i, 1])
      elif len(dup[0]) > 1:
          k = dup[0][0] - 1
          if k != i:
              continue
          else:
              for j in dup[0]:
                  if j != i:
                      inputArr[k, 6] = inputArr[k, 6] + inputArr[j, 6]
              inputArr[k, 6] = inputArr[i, 6] + inputArr[i, 5]
  return(inputArr)


In [12]:
import numpy as np
import pandas as pd
import math as mth
#from Calculation import FWRsweep, BKWsweep

Sb = Sb * 10**6
Vb = Vb * 10**3
Zb = (Vb**2)/Sb

outputArr = np.zeros([branchArr.shape[0], 7], dtype=np.complex_)
busDataArr = np.zeros([busArr.shape[0], busArr.shape[1]], dtype=np.float)

busDataArr[:, 0] = busArr[:, 0]
busDataArr[:, 1] = busArr[:, 1]
busDataArr[:, 2] = busArr[:, 2]
busDataArr[:, 3] = busArr[:, 3]

outputArr[:,0] = branchArr[:,0] # From Node
outputArr[:,1] = branchArr[:,1] # To Node
outputArr[:,2] = branchArr[:,2] # R
outputArr[:,3] = branchArr[:,3] # X
# outputArr[:,4] # Voltage
# outputArr[:,5] # Load Current
# outputArr[:,6] # Current
n = 0 # Number of iterations
Vold = 0 # Old Voltage Value
Vs = 1 # Source Voltage
#Vs = 7200 # Source Voltage

while (1):
    outputArr = FWRsweep.FWR(outputArr, Vs) # Run the forward sweep
    Vl = outputArr[len(outputArr)-1, 4] # The last load on the radial system
    Err = ((abs(Vl.real - Vold.real))/Vs) # Calculate the error in the end
    print("Error value after %s iteration : %f" %(n,Err))
    if Err.real<=Tol:
        print("Final Voltage value :\n%s" %outputArr[:, 4])
        print("Final Current value : \n%s" %outputArr[:, 6])
        print("number of iterations : \n%d" %n)
        break
    else:
        Vold = Vl # Replace the old with the calculated load
        outputArr = BKWsweep.BKWsweep(busDataArr, outputArr) # Run backward sweep
    n += 1
    if n == 50: # If the iteration exceeds 50 iterations, end run
        print("Failed to converge. Report has not been created")
        break
    '''
    BU = busArr[:, 0] #Counts the number of buses
    

    BU = busArr[:,0] #Number of buses
    dfObj = pd.DataFrame(branchArr, columns=['Bus1', 'Bus2', 'R', 'X']) #Create a DataFrame object
    duplicateRowsDF = dfObj[dfObj.duplicated(['Bus1'])]   # Select all duplicate rows based on multiple column names in list
    print("Duplicate Rows based on 2 columns are:", duplicateRowsDF, sep='\n')   # Print all duplicate rows based on multiple column names in list
    I12=I23=I34=I45=I56=I67=I78=I89=I910=0;Vold=0;n=0 ;
    Vs= 7200; # The source voltage at node 1
    while (1):
        FWRsweep(V,Z,I)    # Forward sweep    
        array_length = len(V)
        Vl = V[array_length - 1]
        Err=(abs(Vl.real-Vold.real)/Vs)
        print("Error value after %s iteration : %f" %(n,Err));
        if Err.real<=Tol:
            print("Final Voltage value :\n%s" %V);
            print("Final Current value : \n%s" %Il[:n-1]);
            print("number of iterations : \n%d" %n);
            break;
        else:
        Vold=Vl;   
        BKWsweep(V,Z,I)   # Backward sweep
        n=n+1;
        '''

ModuleNotFoundError: No module named 'Calculation'